In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import argparse
import pickle
import cv2
import os

In [2]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Datasets'))

data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	if label not in LABELS:
		continue
	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

[INFO] loading images...


In [3]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean
n_epochs = 50

## --- Additional part starts here ---

In [9]:
import tensorflow as tf
import tensorflow.compat.v1 as tfcompatv1
import tensorflow_hub as hub

tfcompatv1.disable_eager_execution()
efficientnet_url = "https://tfhub.dev/google/efficientnet/b4/classification/1"

loading_model = hub.Module(efficientnet_url, trainable = True)
# load_model will only have the middle layers of the entire network.
# To use this imported network properly you'll first have to connect your own input layer and output layers like below.

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape = (224, 224, 3)),
    loading_model, # The core part of the network, which we imported above
    # From below, we add additional layers to make our output better
    tf.keras.layers.AveragePooling2D(pool_size = (7, 7)),
    tf.keras.layers.Flatten(name = "flatten"),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(lb.classes_), activation = "softmax")
])

# Now that the model's architecture has been created, freeze it 
for layer in model.layers:
    layer.trainable = False

# ------------------------- Training the model -------------------------
# Compile the model (Can be done only after you freeze the model)
# This is an optimizer similar to SGD that's there in the below code block
adam = tf.keras.optimizers.Adam(lr = 1e-4, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7)
model.compile(loss = "categorical_crossentropy", 
              optimizer = adam, 
              metrics = [accuracy, 
                         tf.keras.metrics.CategoricalAccuracy(), 
                         tf.keras.metrics.TopKCategoricalAccuracy(),
                         tf.keras.metrics.Precision(), 
                         tf.keras.metrics.Recall(), 
                         tf.keras.metrics.TruePositives(), 
                         tf.keras.metrics.TrueNegatives(), 
                         tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])

# Print a summary of the model
model.summary()

history = model.fit(x = trainAug.flow(trainX, trainY, batch_size=32),
        steps_per_epoch = len(trainX) // 32,
        validation_data = valAug.flow(testX, testY),
        validation_steps = len(testX) // 32,
        epochs=n_epochs)

RuntimeError: variable_scope module_2/ was unused but the corresponding name_scope was already taken.

## --- Additional part ends here ---

In [6]:
'''
with tf.device('/gpu:0'):
    # load the ResNet-50 network, ensuring the head FC layer sets are left
    # off
    baseModel = ResNet50(weights="imagenet", include_top=False,
        input_tensor=Input(shape=(224, 224, 3)))

    # construct the head of the model that will be placed on top of the
    # the base model
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(inputs=baseModel.input, outputs=headModel)

    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the training process
    for layer in baseModel.layers:
        layer.trainable = False

    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable)
    print("[INFO] Compiling model...")
    opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / n_epochs)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])
    # train the head of the network for a few epochs (all other layers
    # are frozen) -- this will allow the new FC layers to start to become
    # initialized with actual "learned" values versus pure random
    print("[INFO] Training head...")
    H = model.fit(
        x=trainAug.flow(trainX, trainY, batch_size=32),
        steps_per_epoch=len(trainX) // 32,
        validation_data=valAug.flow(testX, testY),
        validation_steps=len(testX) // 32,
        epochs=n_epochs)
'''

[INFO] Compiling model...
[INFO] Training head...


C:\Users\Yash Umale\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
205/205 [==============================] - 119s 512ms/step - loss: 1.4309 - accuracy: 0.5046 - val_loss: 0.8264 - val_accuracy: 0.6967
Epoch 2/50
205/205 [==============================] - 104s 507ms/step - loss: 0.8809 - accuracy: 0.6794 - val_loss: 0.6620 - val_accuracy: 0.7661
Epoch 3/50
205/205 [==============================] - 103s 503ms/step - loss: 0.7386 - accuracy: 0.7386 - val_loss: 0.5724 - val_accuracy: 0.8024
Epoch 4/50
205/205 [==============================] - 105s 513ms/step - loss: 0.6517 - accuracy: 0.7697 - val_loss: 0.5136 - val_accuracy: 0.8235
Epoch 5/50
205/205 [==============================] - 105s 512ms/step - loss: 0.6065 - accuracy: 0.7881 - val_loss: 0.4645 - val_accuracy: 0.8488
Epoch 6/50
205/205 [==============================] - 99s 482ms/step - loss: 0.5663 - accuracy: 0.8064 - val_loss: 0.4281 - val_accuracy: 0.8571
Epoch 7/50
205/205 [==============================] - 105s 514ms/step - loss: 0.5138 - accuracy: 0.8247 - val_loss: 0.3957 - 

In [7]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = n_epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plotPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Plots'
plt.savefig(plotPath)

[INFO] evaluating network...
              precision    recall  f1-score   support

       Abuse       0.96      0.93      0.95       234
     Assault       0.94      0.51      0.66        57
    Fighting       0.91      0.96      0.93       393
      Normal       1.00      1.00      1.00      1008
     Robbery       0.95      0.98      0.96       333
   Vandalism       0.97      1.00      0.98       172

    accuracy                           0.97      2197
   macro avg       0.95      0.90      0.91      2197
weighted avg       0.97      0.97      0.97      2197

[INFO] Serializing network...


OSError: Unable to create file (unable to open file: name = 'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 302)

In [8]:
# serialize the model to disk
print("[INFO] Serializing network...")
modelPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models\trained_crime'
model.save(modelPath, save_format="h5")

# serialize the label binarizer to disk
lbPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models\lb.pickle'
f = open(lbPath, "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] Serializing network...


C:\Users\Yash Umale\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


## **Video Classification with Rolling Prediction Averaging**

In [3]:
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

from imutils.video import VideoStream
from imutils.video import FPS

In [4]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
lbPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models\lb.pickle'
modelPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models\trained_crime'

model = load_model(modelPath)
lb = pickle.loads(open(lbPath, "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

[INFO] loading model and label binarizer...


In [5]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = VideoStream(src = 0).start()
writer = None
(W, H) = (None, None)

# Change this for different cameras
cameraNo = "A Block" 

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
fps = FPS().start()

# loop over frames from the video file stream
while True:
	# read the next frame from the file
	# (grabbed, frame) = vs.read()
    frame = vs.read()
    
	# if the frame was not grabbed, then we have reached the end
	# of the stream
	# if not grabbed:
	# 	break
        
	# if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    # clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    
    # make predictions on the frame and then update the predictions
	# queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    
	# perform prediction averaging over the current history of
	# previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

	# check if the video writer is None
    outPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Output'
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outPath, fourcc, 30, (W, H), True)
        
	# write the output frame to disk
    cv2.imshow("Frame", output)
    
	# show the output image
    fps.update()
    key = cv2.waitKey(1) & 0xFF
    
	# if the `q` key was pressed, break from the loop
    if (key == ord("q") or key == ord('Q')):
        break

# release the file pointers
print("[INFO] Cleaning up...")
fps.stop()
print("Elapsed time: {:.2f}".format(fps.elapsed()))
print("Approx. FPS: {:.2f}".format(fps.fps()))
writer.release()
vs.stop()

[INFO] Cleaning up...
Elapsed time: 173.63
Approx. FPS: 4.40


In [ ]:
vs = VideoStream(src = 0).start()
writer = None
(W, H) = (None, None)
cameraNo = "A Block" 
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
fps = FPS().start()

while True:
    frame = vs.read()
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

    outPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Output'
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outPath, fourcc, 30, (W, H), True)
        
    cv2.imshow("Frame", output)
    fps.update()

fps.stop()
vs.stop()